# ASFI Fork Merge Analysis

In [45]:
import numpy as np
import pandas as pd
import requests

In [31]:
data=pd.read_csv("../bigfiles/messages_2019_8.csv", usecols=["list","body"], header=0)#, engine="pyarrow")

### Search all message bodies for automated messages signifying a merge between the fork and the main branch

In [34]:
pattern = r'URL:\s*(.*?)\s*This is a PR merged from a forked repository.'
selected=data['body'].str.extract(pattern)
selected=selected.dropna()


317        https://github.com/apache/incubator-pulsar/pul...
356        https://github.com/apache/incubator-airflow/pu...
357        https://github.com/apache/incubator-airflow/pu...
608        https://github.com/apache/incubator-mxnet/pull...
937        https://github.com/apache/incubator-airflow/pu...
                                 ...                        
1383834    https://github.com/apache/incubator-dubbo/pull...
1383874    https://github.com/apache/incubator-dubbo/pull...
1383882    https://github.com/apache/incubator-dubbo/pull...
1383897    https://github.com/apache/incubator-dubbo/pull...
1383927    https://github.com/apache/incubator-dubbo/pull...
Name: 0, Length: 10301, dtype: object

### Get rid of duplicates

In [96]:
pull_list=np.unique(list(selected[0]))

In [ ]:
#Reformat because Github API has changed

In [97]:
urls=[]
for pull_url in pull_list:
    splits=pull_url.split('/')
    splits[2]='api.'+splits[2]
    splits[5]=splits[5]+'s'
    splits.insert(3,'repos')
    url=""

    for split in splits:
        url+=split+'/'
    urls.append(url[:-1])

### Get the originating fork json for each merge

In [98]:
i=0
for url in urls:
    print(url)
    r = requests.get(url)
    print(r.json())
    try:
        if r.json()['message']=="Moved Permanently":
            r = requests.get(r.json()['url'])
        else:
            pass
    except:
        pass
    print(r.json())
    if i>100: #temporary for debugging purposes
        break # main issue is rate limiting - ip rotation or else logging in for higher limit?
    i+=1

https://api.github.com/repos/apache/incubator-airflow-ci/pulls/1
{'url': 'https://api.github.com/repos/apache/airflow-ci/pulls/1', 'id': 215841035, 'node_id': 'MDExOlB1bGxSZXF1ZXN0MjE1ODQxMDM1', 'html_url': 'https://github.com/apache/airflow-ci/pull/1', 'diff_url': 'https://github.com/apache/airflow-ci/pull/1.diff', 'patch_url': 'https://github.com/apache/airflow-ci/pull/1.patch', 'issue_url': 'https://api.github.com/repos/apache/airflow-ci/issues/1', 'number': 1, 'state': 'closed', 'locked': True, 'title': 'Improve Docker image for local development', 'user': {'login': 'seelmann', 'id': 86629, 'node_id': 'MDQ6VXNlcjg2NjI5', 'avatar_url': 'https://avatars.githubusercontent.com/u/86629?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/seelmann', 'html_url': 'https://github.com/seelmann', 'followers_url': 'https://api.github.com/users/seelmann/followers', 'following_url': 'https://api.github.com/users/seelmann/following{/other_user}', 'gists_url': 'https://api.github.com/user

KeyboardInterrupt: 